In [1]:
import numpy as np
import tensorflow as tf

def sigmoid(x):
    return 1/(1+np.exp(-x))

n_inputs = 1000000
n_nodes = 3
n_steps = 2

n_cols = 2

In [2]:
data_x1 = np.array(np.random.rand(n_inputs, n_cols))*2 -1
data_x2 = np.array(np.random.rand(n_inputs, n_cols))*2 -1

In [3]:
state_0 = np.zeros([n_inputs, n_nodes])

W = np.array([[1, 0, -1], [2, 2, 1]])
U = np.array([[1, 1, 0], [0, -2, -1], [-1, 0, 1]])
b = np.array([0.5, -0.5, 1])

WU = np.concatenate([W, U])

def getState(data_x, state, WU, b):
    data = np.concatenate([data_x, state], axis=1)
    return np.tanh(np.matmul(data, WU) + b)

state_1 = getState(data_x1, state_0, WU, b)
state_2 = getState(data_x2, state_1, WU, b)
all_states = np.concatenate([state_1, state_2])



In [4]:
Wo = np.array([[1], [-1], [0.5]])
bo = np.array([-0.5])

final_output = sigmoid(np.matmul(all_states, Wo) + bo)

In [5]:
final_output.shape

(2000000, 1)

In [6]:
inputs = tf.placeholder(tf.float32, shape=[n_steps, n_inputs, n_cols])
inputs_seq = tf.unstack(inputs)
outputs = tf.placeholder(tf.float32, shape=[n_steps*n_inputs, 1])

with tf.name_scope('rnn_cell'):
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_nodes, activation=tf.sigmoid)
    cell_output, state = tf.contrib.rnn.static_rnn(
        cell=basic_cell, inputs=inputs_seq,  dtype=tf.float32)

    all_cell_output = tf.concat(cell_output, axis=0)

In [ ]:
with tf.name_scope("output_layer"):
    output_weights = tf.Variable(tf.truncated_normal([n_nodes, 1]))
    output_bias = tf.Variable(tf.zeros([1]))
    prediction = tf.sigmoid(tf.matmul(all_cell_output, output_weights) + output_bias)
    
with tf.name_scope('evaluation'):
    loss = tf.reduce_mean(tf.square(prediction - outputs))

In [20]:
learning_rate = 0.001

optimaizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train = optimaizer.minimize(loss)

In [22]:
init = tf.global_variables_initializer()
feed_dict = {inputs: np.array([data_x1, data_x2]), outputs: final_output}

epochs = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(epochs):
        sess.run(train, feed_dict)
        if i%10 ==0:
            print(sess.run(loss, feed_dict))
    variables_names =[v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    for k,v in zip(variables_names, values):
        print(k, v)

0.0272215
0.0251074
0.0232011
0.0215216
0.0200732
0.0188465
0.0178238
0.0169824
0.0162969
0.0157425
rnn/basic_rnn_cell/weights:0 [[-0.52484196  0.06126241  0.32770014]
 [ 0.32972711  0.28508118  0.38783094]
 [-0.07141124  0.06239102 -0.48181555]
 [-0.37223026 -0.09905472 -0.27686185]
 [ 0.29076529 -0.71856272 -0.57185042]]
rnn/basic_rnn_cell/biases:0 [-0.07642635  0.08180234 -0.07769082]
output_layer/Variable:0 [[ 0.89362514]
 [-1.64104211]
 [ 1.51233077]]
output_layer/Variable_1:0 [-0.07967644]
